# History matching with RISC

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.ticker import FixedLocator

d = '../../RISC/'

k = 25  # number of ensembles run
sigma = 3
OUTPUTS = ('n small', 'n medium', 'n large')
observations = pd.read_csv(d + 'farm-size-year.csv')
YEARS = range(13)
TOTAL_YEARS = 13
# drivers are in the order given in fig 10 of the paper
drivers = (
          (False, False, False, False), (False, False, False, True),
          (False, False, True, True), (False, False, True, False),
          (False, True, False, True), (False, True, False, False),
          (False, True, True, True), (False, True, True, False),
          (True, False, False, True), (True, False, False, False),
          (True, False, True, True), (True, False, True, False),
          (True, True, False, True), (True, True, False, False),
          (True, True, True, True), (True, True, True, False))


# set automatically
v_ens = 0
m_ens = 0


def get_fp(driver):
    # Get the filepath of a driver
    fp = d + 'results_non_variance/farm-ensembles-'
    fp = fp + 'true-' if driver[0] else fp + 'false-'
    fp = fp + 'true-' if driver[1] else fp + 'false-'
    fp = fp + 'true-' if driver[2] else fp + 'false-'
    fp = fp + 'true-' if driver[3] else fp + 'false-'
    return fp + '1.csv'

The data we are analysing is a time series. We must somehow measure the difference between the estimated time series (from the model outputs) and the expected time series (from the empirical data). I looked at a few different ways of doing this.

Initially, I measured correlation given we're mainly interested in the direction of the trend rather than magnitude of, say, farms of a small size. However, looking at only correlation produced results that poorly reflected the actual variance in the model when variance-rate was included (I'm skipping over the why of this in here for now, but I can discuss it and add it if desired).

I then looked at a few different methods of measuring the size of the error between estimated and expected time series data. These were 
- mean absolute error
- root mean square error
- percentage error, and
- mean absolute scaled error.

I found the final one (mean absolute scaled error) to be the most appropriate. This was a subjective decision, made by visually comparing plots of the time series against the calculated errors, and choosing the method which produced a result that I felt appropriately matched the error I saw in the plot.

In [2]:
def mean_absolute_scaled_error(observations_col, estimates_col):
    """ The observation and estimate must be a single column (output)."""
    result = 0
    # mean absolute error
    mae = np.mean([abs(observations_col[year] - 
                       estimates_col[year])
                    for year in YEARS])
    denom = sum([abs(observations_col[year] - observations_col[year-1])
                 for year in YEARS[1:]]) / (TOTAL_YEARS - 1)
    for year in YEARS:
        obs = observations_col[year]
        est = estimates_col[year]
        error = obs - est
        result += error / denom
    return abs(result / TOTAL_YEARS)

We can use the above to calculate the mean absolute squared error (MASE) between one output of a simulation and the related empirical data, or between two different simulation runs.

The model has 16 different drivers we are performing history matching on.
For each driver we have results from an ensemble of 25 runs.
For a given driver and a given model output, we quantify the magnitude of difference between the 25 ensemble runs using MASE, comparing each run with each other run. We then calculate the variance of these differences to obtain the variance of the ensembles for the given driver and output.
It is common in history matching to take the mean variance across all models (i.e. across all drivers).
We obtain a single measured variance for each model output.

In [3]:
def v_ens_X(driver_indexes):
    """ Calculate the average variance across plausible ensemble runs."""
    results = dict((c, 0) for c in OUTPUTS)
    for di in driver_indexes:
        results_x = v_ens_x(drivers[di])
        for output in OUTPUTS:
            results[output] += results_x[output]
    return dict((c, results[c]/len(driver_indexes)) for c in OUTPUTS)


def v_ens_x(driver):
    """ Calculate the variance of a plausible ensemble."""
    results = dict((c, []) for c in OUTPUTS)
    for output in OUTPUTS:
        r = []
        for i in range(k):
            df = pd.read_csv(get_fp(driver)[:-5] + str(i+1) + '.csv')
            r.append(df[output].tolist())
        for a in range(k-1):
            for b in range(a+1, k):
                results[output].append(max(mean_absolute_scaled_error(r[a], r[b]),
                                        mean_absolute_scaled_error(r[b], r[a])))
    return dict((c, np.var(results[c], ddof=1)) for c in OUTPUTS)

In [4]:
# let's see what the variance is for each column
v_ens_X(range(len(drivers)))

{'n large': 0.035835304631760136,
 'n medium': 0.044756856128314475,
 'n small': 0.03637738203281682}

There's little variance in the ensembles.

Now let's do some history matching. We quantify the implausibility of a driver by measuring the MASE between a single run of the model against the empiricial data for each output. Each MASE (one per output) is then divided by the square root of the ensemble variance for the given output. So, we have a separate measure of implausibility per output.

Generally, if the model is implausible for one output, then we say it is an implausible model, even if its results for other outputs are good.

In [5]:
def implaus(driver):
    """ Calculate the implausiblity of a set of parameters (driver)."""
    estimates = pd.read_csv(get_fp(driver))
    implaus = 0
    for output in OUTPUTS:
        est = estimates[output].tolist()
        obs = observations[output].tolist()
        diff = mean_absolute_scaled_error(obs, est)
        # Take the maximum implausiblity.
        # If the model is implausible for one output
        # then we're not interested in it.
        implaus = max(implaus,
                     diff / np.sqrt(v_ens[output]))
    return round(implaus, 2)


def wave(plaus_space):
    """ Run a single wave of history matching.

        plaus_space: index of plausible drivers to test."""
    globals()['v_ens'] = v_ens_X(plaus_space)
    new_plaus_space = []
    implaus_scores = []
    for i in plaus_space:
        score = implaus(drivers[i])
        implaus_scores.append(score)
        if score < sigma:
            new_plaus_space.append(i)
    return new_plaus_space, implaus_scores


def run_history_matching():
    """ Run waves of history matching until the new plausible space
        is either empty (the whole space is implausible)
        or is unchanged from the previous wave.
    """
    wave_i = 1
    plaus_space = []
    new_plaus_space = range(len(drivers))
    while len(new_plaus_space) > 0 and len(plaus_space) != len(new_plaus_space):
        plaus_space = new_plaus_space
        new_plaus_space, implaus_scores = wave(plaus_space)
        print('Wave ', str(wave_i))
        print('Implausiblity scores: ', implaus_scores)
        print('New plausible drivers:', new_plaus_space)
        print('')
        wave_i += 1


run_history_matching()

Wave  1
Implausiblity scores:  [127.05, 60.78, 51.21, 60.12, 46.58, 50.67, 62.82, 41.28, 155.74, 157.32, 145.71, 147.08, 34.11, 34.43, 34.0, 35.77]
New plausible drivers: []



The plausible space is empty. In general, an implausibility score of 3 is used as a threshold. A score of less than 3 is considered plausible, and more than 3 is considered implausible.

None of the models have obtained a plausible score. This has occurred because there is more uncertainty in the model than we are quantifying. We're only quantifying uncertainty stemming from the variance of the ensembles, but there is very little variance. The models do not perfectly match the empirical data, and we must quantify this error. This is often referred to as *model discrepancy*, *model inadequancy* or *uncertainty in model form*.

We can quantify the model discrepancy by measuring the MASE between each of the 16 possible models and the empirical data. We calculate the average error between the model output and data across all models. We do this separately for each output.

In [6]:
def m_ens_X(plaus_space):
    """ Calculate model discrepancy.
        Calculate the average model discrepency across all plausible models."""
    results = dict((output, 0) for output in OUTPUTS)
    total_models = len(plaus_space)
    for di in plaus_space:
        df = pd.read_csv(get_fp(drivers[di]))
        for output in OUTPUTS:
            est = df[output].tolist()
            obs = observations[output].tolist()
            e = mean_absolute_scaled_error(obs, est)
            results[output] += e
    return dict((c, results[c]/len(plaus_space)) for c in OUTPUTS)

print(m_ens_X(range(len(drivers))))

{'n large': 14.725075204125483, 'n medium': 5.371679110956763, 'n small': 7.079308403460946}


This shows the model discrepancy for each output. The uncertainty in the model's ability to match the empirical data is clearly much higher than the ensemble variance of the models. Therefore, it is more likely to have an effect on the implausibility of the models.

First, we must update the implaus and wave functions to use this new model discrepancy term.

In [7]:
def implaus(driver):
    """ Calculate the implausiblity of a set of parameters (driver)."""
    estimates = pd.read_csv(get_fp(driver))
    implaus = 0
    for output in OUTPUTS:
        est = estimates[output].tolist()
        obs = observations[output].tolist()
        diff = mean_absolute_scaled_error(obs, est)
        # Take the maximum implausiblity.
        # If the model is implausible for one output
        # then we're not interested in it.
        implaus = max(implaus,
                     diff / np.sqrt(v_ens[output] + m_ens[output]))
    return round(implaus, 4)


def wave(plaus_space):
    """ Run a wave of history matching.

        plaus_space: index of plausible drivers to test."""
    globals()['v_ens'] = v_ens_X(plaus_space)
    globals()['m_ens'] = m_ens_X(plaus_space)
    new_plaus_space = []
    implaus_scores = []
    for i in plaus_space:
        score = implaus(drivers[i])
        implaus_scores.append(score)
        if score < sigma:
            new_plaus_space.append(i)
    return new_plaus_space, implaus_scores

run_history_matching()

Wave  1
Implausiblity scores:  [6.2602, 3.3365, 3.2269, 3.3245, 3.0165, 3.0549, 3.1869, 2.9512, 7.6736, 7.7514, 7.1796, 7.2467, 1.6805, 1.6966, 1.6751, 1.7624]
New plausible drivers: [7, 12, 13, 14, 15]

Wave  2
Implausiblity scores:  [4.8274, 2.4754, 2.4991, 2.4674, 2.596]
New plausible drivers: [12, 13, 14, 15]

Wave  3
Implausiblity scores:  [2.522, 2.5461, 2.5139, 2.6448]
New plausible drivers: [12, 13, 14, 15]



Three waves of history matching occurred. In the first, all but drivers 7, 12, 13, 14 and 15 were removed from the plausible space.

In the second wave, driver 7 was removed.

The result from the third wave is the same as the second, so we end the procedure.

We found the final four drivers listed to be plausible. This matches with the results found in the paper.